In [1]:
!pip install pandas
!pip install xgboost
!pip install neurokit2
!pip install polars
!pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requ

In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
import neurokit2 as nk
import polars as pl
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             make_scorer, precision_score, recall_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier as DTC
import os


In [4]:
print("predict_events")
# Define sampling rates
original_sampling = 2000
selected_sampling = 1000

# Open dataset
data = pl.read_csv('C:/Users/eleon/Desktop/SDAP/backend/src/data/p2_wk1-2/0204451eFnorm.csv', columns=['MR', 'ML'])
loc = pl.read_csv('C:/Users/eleon/Desktop/SDAP/backend/src/data/p2_wk1-2/0204451elocation_Bites.csv')

# Get number of sleep cycles (in this case 6)
cycle_num = int(np.ceil(data.shape[0] / original_sampling / 90 / 60))

# Selected 5 min intervals in the sleep stage detection phase
selected = [{"start_id": 600000, "end_id": 1200000},
            {"start_id": 12000000, "end_id": 12600000},
            {"start_id": 12600000, "end_id": 13200000},
            {"start_id": 13800000, "end_id": 14400000},
            {"start_id": 19800000, "end_id": 20400000},
            {"start_id": 22800000, "end_id": 23400000},
            {"start_id": 24000000, "end_id": 24600000},
            {"start_id": 36600000, "end_id": 37200000},
            {"start_id": 40800000, "end_id": 41400000}]



predict_events


# GENERATE MODEL

In [5]:
print("generate_model")

range_min = 0

# Last location end of the test biting
range_max = int(float(loc[2,1]))

# Create array of zeros of the size of the dataset
bites = np.zeros(data.shape[0], dtype=int)
print("label bites")

# Add ones where the test biting were performed
for i in range(1, range_max):
    if i < int(float(loc[0,0])) or (i > int(float(loc[0,1])) and i < int(float(loc[1,0]))) or (i > int(float(loc[1,1])) and i < int(float(loc[2,0]))) or i > int(float(loc[2,1])):
        bites[i] = 0
    else:
        bites[i] = 1

print("add bites")
MR = data['MR']
ML = data['ML']

# Resample signals
MR_resampled = nk.signal_resample(MR, sampling_rate=original_sampling, desired_sampling_rate=selected_sampling, method="pandas")
ML_resampled = nk.signal_resample(ML, sampling_rate=original_sampling, desired_sampling_rate=selected_sampling, method="pandas")
bites = nk.signal_resample(bites, sampling_rate=original_sampling, desired_sampling_rate=selected_sampling, method="pandas")

df = pl.DataFrame({'MR': MR_resampled, 'ML': ML_resampled, 'Bites': bites})

# Data composed of MR and ML
x = df[range_min:range_max,:2].clone()

# Classifications composed from bites
y = df[range_min:range_max,-1].clone()

x = np.array(x)
y = np.array(y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, shuffle=False) # Split data for test and training
SC = StandardScaler()

# Scale features such that they have the properties of a standard normal distribution with a mean of zero and a standard deviation of one
x_train = pd.DataFrame(SC.fit_transform(x_train))
x_test = pd.DataFrame(SC.transform(x_test))


# Fit model
print("fit model")
model = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic',
    eval_metric='logloss', subsample=0.6, max_depth=3, learning_rate=0.1, colsample_bytree=1.0)
model.fit(x_train, y_train)


print("save model")
gen_model = False

# TODO: Insert model info into DB and saves it
model.save_model("/content/drive/My Drive/p2_model.json")


# TODO: check if general model exist in DB
general_model = os.path.exists('/content/drive/My Drive/general_model.json')

if not general_model:
    # TODO: insert general model into DB

    # Why same model variable? Shouldn't they be 2 different models??
    model.save_model('/content/drive/My Drive/general_model.json')
    gen_model = True

"""
# Events predictions
labels = predict_events(DATABASE, model, patient_id, week, night_id, recorder)

# What happens if no predictions are there?
# TODO: check code: is this only on first time?
if (not labels) or (len(labels) < 1):
    with sql.connect(DATABASE) as con:
        cur = con.cursor()
        params = (patient_id)
        query = "DELETE FROM models WHERE patient_id=?"
        cur.execute(query, params)
        # cur.execute(f"DELETE FROM models WHERE patient_id={patient_id}")
        if gen_model:
            cur.execute(f"DELETE FROM models WHERE patient_id=-1")
"""


generate_model
label bites
add bites
fit model
save model


'\n# Events predictions\nlabels = predict_events(DATABASE, model, patient_id, week, night_id, recorder)\n\n# What happens if no predictions are there?\n# TODO: check code: is this only on first time?\nif (not labels) or (len(labels) < 1):\n    with sql.connect(DATABASE) as con:\n        cur = con.cursor()\n        params = (patient_id)\n        query = "DELETE FROM models WHERE patient_id=?"\n        cur.execute(query, params)\n        # cur.execute(f"DELETE FROM models WHERE patient_id={patient_id}")\n        if gen_model:\n            cur.execute(f"DELETE FROM models WHERE patient_id=-1")\n'

#PREDICT EVENTS

In [6]:
# Create new dataframe to locate and add all the selected phases with the indexes
index = [i for i in range(5*original_sampling*60 * len(selected))]

print(len(index))


df = pl.DataFrame()
for i in range(len(selected)):
    #print(data[selected[i]['start_id']:selected[i]['end_id'],:])
    df = pl.concat([df, data[selected[i]['start_id']:selected[i]['end_id'],:]])
#index = df.index

print(df)
#print(index)

5400000
shape: (5_400_000, 2)
┌───────────┬──────────┐
│ MR        ┆ ML       │
│ ---       ┆ ---      │
│ f64       ┆ f64      │
╞═══════════╪══════════╡
│ -2.3222   ┆ -2.8358  │
│ -1.6936   ┆ -2.4757  │
│ -0.95283  ┆ -1.9747  │
│ -0.24272  ┆ -1.419   │
│ …         ┆ …        │
│ -0.023149 ┆ 0.063727 │
│ 0.0041987 ┆ 0.081505 │
│ 0.0094327 ┆ 0.068084 │
│ -0.006336 ┆ 0.043006 │
└───────────┴──────────┘


In [7]:
MR = nk.signal_resample(df.get_column("MR"), sampling_rate=original_sampling, desired_sampling_rate=selected_sampling, method="pandas")
ML = nk.signal_resample(df.get_column("ML"), sampling_rate=original_sampling, desired_sampling_rate=selected_sampling, method="pandas")
df_resampled = pl.DataFrame({'MR': MR, 'ML': ML})

index_resampled = nk.signal_resample(index, sampling_rate=original_sampling, desired_sampling_rate=selected_sampling, method="pandas")

In [8]:
len(index_resampled) == df_resampled['MR'].count()

True

In [9]:
np.array(df_resampled[:,:2].clone().rows()).shape

(2700000, 2)

In [10]:
x = df_resampled[:,:2].clone()
x_p = np.array(x.rows())
y_p = model.predict(x_p)
y_p_mix = []

In [11]:
len(x_p)

2700000

# ELEONORA CODE

In [28]:
predicted_events = []
start = 0
end = 0
first = True
for i in range(len(y_p)):
    if i == len(y_p)-1 and y_p[i] == 1 and y_p[i-1] == 1:
        #print("last number is 1 with 1 as precedent")
        end = i

        duration_s = (end-start)/selected_sampling
        if duration_s >= 0.25:
            predicted_events.append({'location_begin': start, 'location_end': end, 'duration_s': (end-start)*selected_sampling*60})

    if i == len(y_p)-1 and y_p[i] == 1 and y_p[i-1] == 0:
        break

    if i == len(y_p)-1 and y_p[i] == 0:
        break

    if y_p[i] == 1 and first == True:
        #print("number is the first one: start event")
        start = i
        first = False

    if y_p[i+1] == 1:
       # print("next number is one as well, continue")
        continue

    if y_p[i] == 1 and first == False and y_p[i+1] == 0:
        #print("next number is 0, current number is 1: end event")
        end = i

        duration_s = (end-start)/selected_sampling
        if duration_s >= 0.25:
            predicted_events.append({'location_begin': start, 'location_end': end, 'duration_s': duration_s})
        first = True

    if y_p[i] == 0:
        #print("number is 0")
        continue




In [29]:
predicted_events

[{'location_begin': 300084, 'location_end': 300681, 'duration_s': 0.597},
 {'location_begin': 301523, 'location_end': 302180, 'duration_s': 0.657},
 {'location_begin': 303122, 'location_end': 303759, 'duration_s': 0.637},
 {'location_begin': 304721, 'location_end': 305438, 'duration_s': 0.717},
 {'location_begin': 306440, 'location_end': 307138, 'duration_s': 0.698},
 {'location_begin': 308239, 'location_end': 308937, 'duration_s': 0.698},
 {'location_begin': 309938, 'location_end': 310575, 'duration_s': 0.637},
 {'location_begin': 311317, 'location_end': 311914, 'duration_s': 0.597},
 {'location_begin': 312816, 'location_end': 313453, 'duration_s': 0.637},
 {'location_begin': 314195, 'location_end': 314832, 'duration_s': 0.637},
 {'location_begin': 315594, 'location_end': 316271, 'duration_s': 0.677},
 {'location_begin': 317153, 'location_end': 317870, 'duration_s': 0.717},
 {'location_begin': 318752, 'location_end': 319469, 'duration_s': 0.717},
 {'location_begin': 320291, 'location_

In [30]:
len(predicted_events)

152

# YIFAN CODE

In [ ]:
# Perform prediction on selected labels
x = df_resampled[:,:2].clone()
x_p = np.array(x.rows())
y_p = model.predict(x_p)

# What is y_p mix?
y_p_mix = []
i = 0
event = 0
loc_start = 0
loc_end = 0

predicted_labels = {}

while i < len(y_p):
    # What is y_sum?
    y_sum = y_p[i];
    cnt = 1;

    # What are you looping in here?
    for j in range(1, 5*selected_sampling+1):
        # What check is performed here? Why 5*original_sampling?
        if(i+j >= len(y_p) or ((index[i+j] - index[i])>5*original_sampling)):
            #print("dur:", cnt/selected_sampling)
            break;
        y_sum += y_p[i+j];
        cnt += 1;

    # What are you checking with this 0.6?
    if y_sum/cnt >= 0.6:
        # Same here: why 55*selected_sampling?
        if(loc_start == 0 or loc_end-loc_start > 55*selected_sampling):
            event += 1
            loc_start = i
            loc_end = i+cnt-1
            location_begin = index[i]
            location_end = index[loc_end]
            # why are you saving location_begin and end, and start_index and end, which are the same?
            predicted_labels[event] = {'location_begin': location_begin, 'location_end': location_end, 'start_index': i, 'end_index': loc_end}

            cycle = np.floor(location_begin / 90 / 60 / original_sampling)

        # What are you checking here? Why are you updating?
        elif(y_p_mix[-1] == 10):
            loc_end += cnt-1
            predicted_labels[event].update({'location_end': index[loc_end], 'end_index': loc_end})
            print("update end:", loc_end)
            print("update end idx:", index[loc_end])
        # What is this for?
        y_p_mix.extend([10]*cnt)
    else:
        loc_start = 0
        # Same here
        y_p_mix.extend([0]*cnt)
    i += cnt

In [37]:
predicted_labels

{1: {'location_begin': 300060,
  'location_end': 360060,
  'start_index': 300060,
  'end_index': 360060},
 2: {'location_begin': 360072,
  'location_end': 420072,
  'start_index': 360072,
  'end_index': 420072},
 3: {'location_begin': 420084,
  'location_end': 480084,
  'start_index': 420084,
  'end_index': 480084},
 4: {'location_begin': 480096,
  'location_end': 540096,
  'start_index': 480096,
  'end_index': 540096},
 5: {'location_begin': 540108,
  'location_end': 600108,
  'start_index': 540108,
  'end_index': 600108},
 6: {'location_begin': 600120,
  'location_end': 655120,
  'start_index': 600120,
  'end_index': 655120},
 7: {'location_begin': 660132,
  'location_end': 670132,
  'start_index': 660132,
  'end_index': 670132},
 8: {'location_begin': 880176,
  'location_end': 890176,
  'start_index': 880176,
  'end_index': 890176}}

In [36]:
"""
# Perform prediction on selected labels
x = df_resampled[:,:2].clone()
x_p = np.array(x.rows())
y_p = model.predict(x_p)


y_p_mix = []
i = 0
event = 0
loc_start = 0
loc_end = 0

predicted_labels = {}

while i < len(y_p):
    y_sum = y_p[i];
    cnt = 1;
    for j in range(1, 5*selected_sampling+1):
        if(i+j >= len(y_p) or ((index[i+j] - index[i])>5*original_sampling)):
            # print("dur:", cnt/selected_sampling)
            break;
        y_sum += y_p[i+j];
        cnt += 1;
    if y_sum/cnt >= 0.6:
        if(loc_start == 0 or loc_end-loc_start > 55*selected_sampling):
            event += 1
            loc_start = i
            loc_end = i+cnt-1
            location_begin = index[i]
            location_end = index[loc_end]
            #params = (patient_id, week, night_id, recorder, event, location_begin, location_end, i, loc_end)
            #print(params)
            #query = "INSERT INTO predicted_labels (patient_id, week, night_id, recorder, label_id, location_begin, location_end, start_index, end_index) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"
            #cur.execute(query, params)

            predicted_labels[event] = {'location_begin': location_begin, 'location_end': location_end, 'start_index': i, 'end_index': loc_end}

            cycle = np.floor(location_begin / 90 / 60 / original_sampling)
            #params = (patient_id, week, night_id, cycle)
            #query = "SELECT count,type from week_summary WHERE (patient_id=? AND week=? AND night_id=? AND cycle=?)"
            #cur.execute(query, params)
            #count = cur.fetchall()

            if count:
                # print(count[0][0])
                if cycle < cycle_num/2:
                    type_ind = 0
                else:
                    type_ind = count[0][1] + 1
                params = (count[0][0]+1, type_ind, patient_id, week, night_id, cycle)
                query = "UPDATE week_summary SET count = ?, type = ? WHERE (patient_id=? AND week=? AND night_id=? AND cycle=?)"
                cur.execute(query, params)
            else:
                if cycle < cycle_num/2:
                    type_ind = 0
                else:
                    type_ind = 1
                params = (patient_id, week, night_id, cycle, cycle_num, 1, type_ind)
                query = "INSERT INTO week_summary (patient_id, week, night_id, cycle, max_cycle, count, type) VALUES (?, ?, ?, ?, ?, ?, ?)"
                cur.execute(query, params)
            print("current end idx:", location_end)

        elif(y_p_mix[-1] == 10):
            loc_end += cnt-1
            #params = (index[loc_end], loc_end, patient_id, week, night_id, recorder, event)
            #query = "UPDATE predicted_labels SET location_end = ?, end_index = ? WHERE (patient_id=? AND week=? AND night_id=? AND recorder=? AND label_id=?)"
            #cur.execute(query, params)
            predicted_labels[event].update({'location_end': index[loc_end], 'end_index': loc_end})
            print("update end:", loc_end)
            print("update end idx:", index[loc_end])
        y_p_mix.extend([10]*cnt);
    else:
        loc_start = 0
        y_p_mix.extend([0]*cnt);
    i += cnt;
    # print(i)
# print(event)
#params = (patient_id, week, night_id, recorder)
#query = "SELECT DISTINCT * from predicted_labels WHERE (patient_id=? AND week=? AND night_id=? AND recorder=?)"
#labels = cur.execute(query, params).fetchall()
#return labels
"""


'\n# Perform prediction on selected labels\nx = df_resampled[:,:2].clone()\nx_p = np.array(x.rows())\ny_p = model.predict(x_p)\n\n\ny_p_mix = []\ni = 0\nevent = 0\nloc_start = 0\nloc_end = 0\n\npredicted_labels = {}\n\nwhile i < len(y_p):\n    y_sum = y_p[i];\n    cnt = 1;\n    for j in range(1, 5*selected_sampling+1):\n        if i == 0:\n            print(f\'j :{j} i+j: {i+j}\')\n        if(i+j >= len(y_p) or ((index[i+j] - index[i])>5*original_sampling)):\n            if i== 0:\n                print("dur:", cnt/selected_sampling)\n            break;\n        y_sum += y_p[i+j];\n        cnt += 1;\n    if y_sum/cnt >= 0.6:\n        if(loc_start == 0 or loc_end-loc_start > 55*selected_sampling):\n            event += 1\n            loc_start = i\n            loc_end = i+cnt-1\n            location_begin = index[i]\n            location_end = index[loc_end]\n            #params = (patient_id, week, night_id, recorder, event, location_begin, location_end, i, loc_end)\n            #print